In [2]:
import glob
from tqdm.auto import tqdm, trange
import requests
import xml.etree.ElementTree as et
import json
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt
import random
import time

In [3]:
from html_parser import WykopParser

In [11]:
tag ='polska'
total_pages = 100


results = []
bar = trange(1, total_pages + 1)
for page_n in bar:
    success = False
    while not success:
        try:
            r = requests.get(f'https://www.wykop.pl/strona/{page_n}/').text
            success = True
        except:
            time.sleep(1)
    
    parser = WykopParser()
    parser.feed(r)
    
    articles = parser.root\
                     .find('div', [('class', 'article'), ('class', 'clearfix'), ('class', 'preview'), ('class', 'dC')])\
                     .contain('a', [('class', 'tag'), ('class', 'affect'), ('class', 'create')], [tag])
        
    titles= articles.find('h2')\
                    .find('a', [('href', 'http'), ('title', '')]) 
    texts = articles.find('p', [('class', 'text')])\
                    .find('a', [('href', 'http')])
    times = articles.find('time', [('datetime', '')])
    
    if len(texts) != len(titles) or len(times) != len(titles):
        print('broken page')
        continue
    
    for i, a1 in enumerate(titles):
        results.append((times[i].attrs['datetime'], a1.attrs['href'], a1.data, texts[i].data))
        
    bar.set_description('Scraped wykops: '+str(len(results)))
    

In [12]:
results[-1]

('2020-11-07T16:26:01+01:00',
 'https://www.wykop.pl/link/5787203/minister-ka-z-pis-zignorowala-caly-szereg-przepisow-prawa-pracy-sady-mowia-nie/',
 'Minister(ka) z PIS zignorowała cały szereg przepisów prawa pracy Sądy mówią NIE',
 '\n\t\t\t\t\tWiceminister sprawiedliwości w piśmie do prezesów sądów rekomenduje pracę na dwie zmiany. I to już od poniedziałku. Niestety poza poleceniem sądy nie otrzymały instrukcji jak zgodnie z prawem wprowadzić tego rodzaju zmiany oraz czy minister oczekuje łamania prawa pracy. Więcej w pierwszym wpisie.\t\t        ')